<a href="https://colab.research.google.com/github/armstrohiro/Pose-Estimations/blob/main/mediapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MediaPipe

*   2/22日用
*   関節座標データ



In [ ]:
!pip install mediapipe

import numpy as np
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
import pandas as pd
from matplotlib.animation import FuncAnimation
import numpy as np
from IPython.display import HTML
from IPython.display import IFrame
from IPython.display import display
!pip install youtube_dl
import youtube_dl

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.2 MB/s eta 0:00:00


In [ ]:
# 動画ファイル読み込み（/content/に動画を入れる）
input_filename = '00001.mp4'
video_data = cv2.VideoCapture(input_filename)
video_width = int(video_data.get(cv2.CAP_PROP_FRAME_WIDTH))
video_hight = int(video_data.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('FPS:',video_data.get(cv2.CAP_PROP_FPS))
print('Dimensions:',video_width,video_hight)

video_data_array = []

print("VideoFrame:",int(video_data.get(cv2.CAP_PROP_FRAME_COUNT)))

#ファイルが正常に読み込めている間ループする
while video_data.isOpened():
  #1フレームごとに読み込み
  success, image = video_data.read()
  if success:
    #フレームの画像を追加
    video_data_array.append(image)
  else:
    break
video_data.release()
print('Frames Read:',len(video_data_array))

FPS: 0.0
Dimensions: 0 0
VideoFrame: 0
Frames Read: 0


## 2/22日用


1. 一つずつ



In [ ]:
import cv2
import mediapipe as mp
import os
from IPython.display import HTML
from base64 import b64encode

# MediaPipe のセットアップ
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 動画ファイルのパス（/content にある動画）
input_filename = "/content/videos/00001.mp4"
output_video_path = "/content/output_00001.mp4"  # 出力動画

# 動画を読み込む
video_data = cv2.VideoCapture(input_filename)
video_width = int(video_data.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_data.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_data.get(cv2.CAP_PROP_FPS))

print(f"FPS: {fps}")
print(f"Dimensions: {video_width} x {video_height}")

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (video_width, video_height))

# MediaPipe Pose の初期化
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while video_data.isOpened():
        success, frame = video_data.read()
        if not success:
            break

        # BGR → RGB に変換（MediaPipe 用）
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # 姿勢推定を実行
        results = pose.process(image)

        # RGB → BGR に戻す（OpenCV 用）
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 骨格を描画
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing_styles.get_default_pose_landmarks_style()
            )

        # 処理したフレームを動画に保存
        out.write(image)

# 終了処理
video_data.release()
out.release()

FPS: 30
Dimensions: 640 x 360


KeyboardInterrupt: 



2. まとめて



In [ ]:
#全部
import cv2
import mediapipe as mp
import os
from IPython.display import display, HTML
from base64 import b64encode
from glob import glob

# MediaPipe のセットアップ
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 解析する動画を /content/ から自動取得
input_videos = glob("/content/videos/*.mp4")  # /content/ 内のすべての .mp4 ファイルを取得
output_videos = []

if not input_videos:
    print("エラー: /content/videos に .mp4 ファイルが見つかりません！動画をアップロードしてください。")
else:
    print(f"{len(input_videos)} 本の動画を解析します: {input_videos}")

    # MediaPipe Pose を初期化
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for input_filename in input_videos:
            output_filename = input_filename.replace(".mp4", "_output.mp4")
            output_videos.append(output_filename)

            # 動画の読み込み
            video_data = cv2.VideoCapture(input_filename)
            video_width = int(video_data.get(cv2.CAP_PROP_FRAME_WIDTH))
            video_height = int(video_data.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(video_data.get(cv2.CAP_PROP_FPS))

            print(f"\n解析中: {input_filename} (FPS: {fps}, サイズ: {video_width}x{video_height})")

            # 出力動画の設定
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_filename, fourcc, fps, (video_width, video_height))

            while video_data.isOpened():
                success, frame = video_data.read()
                if not success:
                    break

                # BGR → RGB に変換（MediaPipe 用）
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # 姿勢推定を実行
                results = pose.process(image)

                # RGB → BGR に戻す（OpenCV 用）
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # 骨格を描画
                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing_styles.get_default_pose_landmarks_style()
                    )

                # 処理したフレームを動画に保存
                out.write(image)

            # 終了処理
            video_data.release()
            out.release()
            print(f"保存完了: {output_filename}")

In [ ]:
#動画が再生されない
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/processed_00001.mp4', 'rb').read()
video_src = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
    <video width="432" height="324" controls>
        <source type="video/mp4" src="{video_src}">
    </video>
""")

## 関節座標出力コード

In [ ]:
# 動画ファイル読み込み（/contentに動画を入れる）
input_filename = '00015.mp4'
video_data = cv2.VideoCapture(input_filename)
video_width = int(video_data.get(cv2.CAP_PROP_FRAME_WIDTH))
video_hight = int(video_data.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('FPS:',video_data.get(cv2.CAP_PROP_FPS))
print('Dimensions:',video_width,video_hight)

video_data_array = []

print("VideoFrame:",int(video_data.get(cv2.CAP_PROP_FRAME_COUNT)))

#ファイルが正常に読み込めている間ループする
while video_data.isOpened():
  #1フレームごとに読み込み
  success, image = video_data.read()
  if success:
    #フレームの画像を追加
    video_data_array.append(image)
  else:
    break
video_data.release()
print('Frames Read:',len(video_data_array))

In [ ]:
with mp_pose.Pose(
  static_image_mode=False,
  model_complexity=2,
  enable_segmentation=True,
  min_detection_confidence=0.5) as pose:

  #動画データをループ処理
  for loop_counter,image_data in enumerate(video_data_array):

    #画像解析
    results = pose.process(cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
      continue

    #解析結果を動画に描画
    mp_drawing.draw_landmarks(
        image_data,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

plt.imshow(cv2.cvtColor(video_data_array[0], cv2.COLOR_BGR2RGB))
plt.tight_layout()

In [ ]:
import csv

# 必要な関節を指定（右から左の順序にし、hipは手首と膝の間に挿入）
target_landmarks = [
    mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.LEFT_EYE,
    mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_SHOULDER,
    mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.LEFT_ELBOW,
    mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.LEFT_WRIST,
    # hipは手首と膝の間に挿入するため、ここには含めない
    mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.LEFT_KNEE,
    mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_FOOT_INDEX
]

# CSVファイルの列名を定義（右から左、hipは手首と膝の間に挿入）
landmark_names = []
for landmark in target_landmarks:
    if landmark == mp_pose.PoseLandmark.RIGHT_WRIST:  # 右手首の後にhipを追加
        landmark_names.append('hip_x')
        landmark_names.append('hip_y')
    landmark_names.append(f'{landmark.name}_x')
    landmark_names.append(f'{landmark.name}_y')

# CSVファイルの書き込み設定
csv_filename = 'landmarks_00015.mp4.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    # ヘッダーを書き込み（フレーム番号と指定した関節の情報）
    writer.writerow(['frame'] + landmark_names)

    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:

        # 動画データをループ処理
        for frame_number, image_data in enumerate(video_data_array):
            # 画像の高さと幅を取得
            height, width, _ = image_data.shape

            # 画像解析
            results = pose.process(cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB))
            if not results.pose_landmarks:
                continue

            # 各指定したランドマークの座標を抽出
            row = [frame_number]  # フレーム番号を追加
            for landmark in target_landmarks:
                # 手首の後にhipを追加
                if landmark == mp_pose.PoseLandmark.RIGHT_WRIST:
                    left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
                    right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
                    hip_x = (left_hip.x + right_hip.x) / 2 * width
                    hip_y = (left_hip.y + right_hip.y) / 2 * height
                    row.extend([hip_x, hip_y])

                # 各関節の x, y 座標を取得し、ピクセル単位に変換
                lm = results.pose_landmarks.landmark[landmark]
                x_px = lm.x * width         # x座標をピクセル単位に変換（小数）
                y_px = lm.y * height        # y座標をピクセル単位に変換（小数）
                row.extend([x_px, y_px])    # x, yを交互に追加

            # CSVファイルに行を書き込み
            writer.writerow(row)

print(f'Specified landmark coordinates saved to {csv_filename}')


In [ ]:
#全部まとめて
import csv
import cv2
import os

# 必要な関節を指定（右から左の順序にし、hipは手首と膝の間に挿入）
target_landmarks = [
    mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.LEFT_EYE,
    mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_SHOULDER,
    mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.LEFT_ELBOW,
    mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.LEFT_WRIST,
    # hipは手首と膝の間に挿入するため、ここには含めない
    mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.LEFT_KNEE,
    mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_FOOT_INDEX
]

# CSVファイルの列名を定義（右から左、hipは手首と膝の間に挿入）
landmark_names = []
for landmark in target_landmarks:
    if landmark == mp_pose.PoseLandmark.RIGHT_WRIST:  # 右手首の後にhipを追加
        landmark_names.append('hip_x')
        landmark_names.append('hip_y')
    landmark_names.append(f'{landmark.name}_x')
    landmark_names.append(f'{landmark.name}_y')

# MediaPipeの姿勢推定設定
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:

    # 00001.mp4から00015.mp4までを処理
    for video_num in range(1, 14):
        input_filename = f'{video_num:05}.mp4'
        video_data = cv2.VideoCapture(input_filename)

        # 動画が存在しない場合はスキップ
        if not video_data.isOpened():
            print(f"Cannot open video file: {input_filename}")
            continue

        # 動画の出力用CSVファイル名
        csv_filename = f'landmarks_{input_filename}.csv'
        with open(csv_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            # ヘッダーを書き込み（フレーム番号と指定した関節の情報）
            writer.writerow(['frame'] + landmark_names)

            frame_number = 0
            while video_data.isOpened():
                # 各フレームを読み込み
                success, image_data = video_data.read()
                if not success:
                    break

                # 画像の高さと幅を取得
                height, width, _ = image_data.shape

                # 画像解析
                results = pose.process(cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB))
                if not results.pose_landmarks:
                    frame_number += 1
                    continue

                # 各指定したランドマークの座標を抽出
                row = [frame_number]  # フレーム番号を追加
                for landmark in target_landmarks:
                    # 手首の後にhipを追加
                    if landmark == mp_pose.PoseLandmark.RIGHT_WRIST:
                        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
                        right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
                        hip_x = (left_hip.x + right_hip.x) / 2 * width
                        hip_y = (left_hip.y + right_hip.y) / 2 * height
                        row.extend([hip_x, hip_y])

                    # 各関節の x, y 座標を取得し、ピクセル単位に変換
                    lm = results.pose_landmarks.landmark[landmark]
                    x_px = lm.x * width         # x座標をピクセル単位に変換（小数）
                    y_px = lm.y * height        # y座標をピクセル単位に変換（小数）
                    row.extend([x_px, y_px])    # x, yを交互に追加

                # CSVファイルに行を書き込み
                writer.writerow(row)
                frame_number += 1

        video_data.release()
        print(f'Specified landmark coordinates saved to {csv_filename}')
